# Drude Oscillator Example
## This example simply illustrates that the Drude Oscillator polarizable force field can be utizied in pyCHARMM.
## Example taken from c43test/lpcent.inp

In [1]:
# The purpose of this script is to illustrate the use of some Drude oscillator
# functionality in pyCHARMM. This is a redo of the test case c43test/lpcent.inp
#  copyright C.L. Brooks III, June 20, 2022

# These are general python modules needed for this  tutorial
import os
import sys

# These are a subset of the pycharmm modules that were installed when
# pycharmm was installed in your python environment
import pycharmm
import pycharmm.generate as gen
import pycharmm.ic as ic
import pycharmm.coor as coor
import pycharmm.energy as energy
import pycharmm.dynamics as dyn
import pycharmm.nbonds as nbonds
import pycharmm.minimize as minimize
import pycharmm.crystal as crystal
import pycharmm.image as image
import pycharmm.psf as psf
import pycharmm.read as read
import pycharmm.write as write
import pycharmm.settings as settings
import pycharmm.cons_harm as cons_harm
import pycharmm.cons_fix as cons_fix
import pycharmm.select as select
import pycharmm.shake as shake

from pycharmm.lib import charmm as libcharmm


In [4]:
# stream in the drude rtf/parametter files using the pycharmm.lingo module
# equivalent to the CHARMM scripting command: stream toppar/toppar_drude.str
read.stream('../toppar/toppar_drude.str')

  
 CHARMM>     stream ../toppar/toppar_drude.str
 VOPEN> Attempting to open::../TOPPAR/TOPPAR_DRUDE.STR::
 OPNLGU> Unit 99 opened for READONLY access to ../TOPPAR/TOPPAR_DRUDE.STR

                    INPUT STREAM SWITCHING TO UNIT    99
 RDTITL> * DRUDE CATION-PI MODEL COMPOUNDS
 RDTITL> * REFERENCES:
 RDTITL> * THIS WORK:
 RDTITL> *    ORABI, LOPES, MACKERELL, AND LAMOUREUX, SUBMITTED
 RDTITL> * FOR SWM4:
 RDTITL> *    LAMOUREUX, HARDER, VOROBYOV, ROUX, AND MACKERELL, JR. CHEM. PHYS. LETT. 2006, 418, 245-249
 RDTITL> * FOR BENX AND NH4+:
 RDTITL> *    ORABI AND LAMOUREUX. J. CHEM. THEORY COMPUT. 2012, 8, 182-193
 RDTITL> * FOR NH4+ AND AROMATICS:
 RDTITL> *    WANG, ORABI, BADAY, BERNECHE, AND LAMOUREUX. J. AM. CHEM. SOC. 2012, USA 134, 10419-1042
 RDTITL> *
 Parameter: IN1 <- "" <empty>
  
 CHARMM>     
  
 CHARMM>    ioformat extended
 MISCOM> Expanded I/O format is used.
  
 CHARMM>     
  
 CHARMM>    bomlev -1
  
 CHARMM>     
  
 CHARMM>    read rtf card !append
 MAINIO> Resid

In [5]:
import numpy as np
read.sequence_string('BENX')

# equivalent to the CHARMM scripting command: generate ADP first ACE last CT3 setup
gen.new_segment(seg_name='BENX', 
                setup_ic=True, warn=True,
                drude=True, mass=0.4, angle=True, dihedral=True)
lpbx = pycharmm.SelectAtoms(atom_type='LPBX')
lpbx_present = np.sum(np.array(list(lpbx))) > 0
print('testcase result: {} -- reading'.format(lpbx_present))


          RESIDUE SEQUENCE --     1 RESIDUES
          BENX    
 NO PATCHING WILL BE DONE ON THE FIRST RESIDUE
 NO PATCHING WILL BE DONE ON THE LAST  RESIDUE
 AUTGEN: Autogenerating specified angles and dihedrals.

 MKDRUDE generate list and setup for drude polarizability

 All okay for      6 added Drude particles 

 GENPSF> Segment   1 has been generated. Its identifier is BENX.
 PSFSUM> PSF modified: NONBOND lists and IMAGE atoms cleared.
 PSFSUM> Summary of the structure file counters :
         Number of segments      =        1   Number of residues   =        1
         Number of atoms         =       19   Number of groups     =        1
         Number of bonds         =       18   Number of angles     =       18
         Number of dihedrals     =       24   Number of impropers  =        0
         Number of cross-terms   =        0   Number of autogens   =        0
         Number of HB acceptors  =        0   Number of HB donors  =        0
         Number of NB exclusions = 

In [6]:
coor.show()
ic.prm_fill(replace_all=True)
ic.seed(res1=1, atom1='CD1', res2=1, atom2='CE1', res3=1, atom3='CZ')
ic.build()
pycharmm.lingo.charmm_script('coor sdrude')
pycharmm.lingo.charmm_script('coor shake')
coor.show()


          COORDINATE FILE MODULE
 TITLE>  * EXECUTING CHARMM SCRIPT FROM PYTHON
 TITLE>  *
        19  EXT
         1         1  BENX      CG           9999.0000000000     9999.0000000000     9999.0000000000  BENX      1               0.0000000000
         2         1  BENX      DCG          9999.0000000000     9999.0000000000     9999.0000000000  BENX      1               0.0000000000
         3         1  BENX      HG           9999.0000000000     9999.0000000000     9999.0000000000  BENX      1               0.0000000000
         4         1  BENX      CD1          9999.0000000000     9999.0000000000     9999.0000000000  BENX      1               0.0000000000
         5         1  BENX      DCD1         9999.0000000000     9999.0000000000     9999.0000000000  BENX      1               0.0000000000
         6         1  BENX      HD1          9999.0000000000     9999.0000000000     9999.0000000000  BENX      1               0.0000000000
         7         1  BENX      CD2          9

1

In [7]:
if not os.path.isdir('pdb'): os.system('mkdir pdb')
write.coor_pdb('pdb/lpcent.pdb')
write.psf_card('pdb/lpcent.psf')
psf.delete_atoms(pycharmm.SelectAtoms().all_atoms())

  
 CHARMM>     write name pdb/lpcent.pdb -
 CHARMM>     coor pdb
 VOPEN> Attempting to open::pdb/lpcent.pdb::
 RDTITL>  
 RDTITL> No title read.
  Write CHARMM-pdb format
 VCLOSE: Closing unit   91 with status "KEEP"
  
 CHARMM>     
  
  
 CHARMM>     write name pdb/lpcent.psf -
 CHARMM>     psf card
 VOPEN> Attempting to open::pdb/lpcent.psf::
 RDTITL>  
 RDTITL> No title read.
 VCLOSE: Closing unit   91 with status "KEEP"
 VCLOSE: Closing unit   91 with status "KEEP"
  
 CHARMM>     
  

 Message from MAPIC: Atom numbers are changed.
 LONEPRD:    1 lonepair atoms deleted

 Message from MAPIC:          1 residues deleted.

 Message from MAPIC:          1 segments deleted.
 DELTIC:        18 bonds deleted
 DELTIC:        18 angles deleted
 DELTIC:        24 dihedrals deleted


True

In [8]:
read.psf_card('pdb/lpcent.psf')
read.pdb('pdb/lpcent.pdb')
lpbx = pycharmm.SelectAtoms(atom_type='LPBX')
lpbx_present = np.sum(np.array(list(lpbx))) > 0
print('testcase result: {} -- writing'.format(lpbx_present))

  
 CHARMM>     read psf card -
 CHARMM>     name pdb/lpcent.psf
 VOPEN> Attempting to open::PDB/LPCENT.PSF::
 MAINIO> Protein structure file being read from unit  91.
 psf_read_formatted: Reading PSF in the expanded format.
 TITLE>  * EXECUTING CHARMM SCRIPT FROM PYTHON
 TITLE>  *  DATE:     1/ 8/23     19:13:44      CREATED BY USER: brookscl
 TITLE>  *
 PSFSUM> PSF modified: NONBOND lists and IMAGE atoms cleared.
 PSFSUM> Summary of the structure file counters :
         Number of segments      =        1   Number of residues   =        1
         Number of atoms         =       19   Number of groups     =        1
         Number of bonds         =       18   Number of angles     =       18
         Number of dihedrals     =       24   Number of impropers  =        0
         Number of cross-terms   =        0   Number of autogens   =        0
         Number of HB acceptors  =        0   Number of HB donors  =        0
         Number of NB exclusions =        0   Total charge =   